#  <font color='red'>Set up Colab Environment</font>

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import os
os.chdir('/content/gdrive/My Drive/Mercari2')
print("We are currently in the folder of ",os.getcwd())

We are currently in the folder of  /content/gdrive/My Drive/Mercari2


#  <font color='red'>Function 1</font>

In [5]:
'''
This function takes 'df_test' as input and gives out the predicted price 'y' value(float).

'df_test' is a data point takes the Inputs of 
["train_id" (index),
"name" (str),
"item_condition_id" (categorical),
"category_name" (str),
"brand_name" (str),
"shipping" (categorical),
"item_description" (str)]
'''

'\nThis function takes \'df_test\' as input and gives out the predicted price \'y\' value(float).\n\n\'df_test\' is a data point takes the Inputs of \n["train_id" (index),\n"name" (str),\n"item_condition_id" (categorical),\n"category_name" (str),\n"brand_name" (str),\n"shipping" (categorical),\n"item_description" (str)]\n'

In [0]:
# ref - https://www.kaggle.com/lopuhin/mercari-golf-0-3875-cv-in-75-loc-1900-s

def data_preprocess(df):
    df['name'] = df['name'].fillna('') + ' ' + df['brand_name'].fillna('')
    df['text'] = (df['item_description'].fillna('') + ' ' + df['name'] + ' ' + df['category_name'].fillna(''))
    return df[['name', 'text', 'shipping', 'item_condition_id']]

In [0]:
def text_encoder(train,test,vect_type,params):
    vectorizer = CountVectorizer(ngram_range = params[0],min_df = params[1],max_df = params[2],max_features = params[3],token_pattern = '\w+',
                                 dtype = np.float32) if vect_type == 'BOW' else TfidfVectorizer(ngram_range = params[0],min_df = params[1],
                                                                                                max_df = params[2],max_features = params[3],token_pattern = '\w+',
                                                                                                dtype = np.float32)
    train_transform = vectorizer.fit_transform(train)
    test_transform = vectorizer.transform(test)
    feat_names = vectorizer.get_feature_names()
    return train_transform,test_transform

In [0]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

def text_preprocess(data):
    preprocessed = []
    # tqdm is for printing the status bar
    for sentance in tqdm_notebook(data):
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\"', ' ')
        sent = sent.replace('\\n', ' ')
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # https://gist.github.com/sebleier/554280
        sent = ' '.join(e for e in sent.split() if e not in stopwords)
        preprocessed.append(sent.lower().strip())
    return preprocessed

In [0]:
def dummy_encoder(train,test):
    enc1 = CountVectorizer(vocabulary= list(train['shipping'].unique()), lowercase=False, binary=True)
    train_transform1 = enc1.fit_transform(train['shipping'].astype(str))
    test_transform1 = enc1.transform(test['shipping'].astype(str))

    enc2 = CountVectorizer(vocabulary= list(train['item_condition_id'].unique()), lowercase=False, binary=True)
    train_transform2 = enc2.fit_transform(train['item_condition_id'].astype(str))
    test_transform2 = enc2.transform(test['item_condition_id'].astype(str))   

    train_transform = scipy.sparse.hstack((train_transform1,train_transform2)).tocsr()
    test_transform = scipy.sparse.hstack((test_transform1,test_transform2)).tocsr()
    
    return train_transform,test_transform

In [0]:
def category_encoder(train,test):
    unique_categories = pd.Series("/".join(train["category_name"].unique().astype("str")).split("/")).unique()
    count_category = CountVectorizer(vocabulary = unique_categories,lowercase = False,binary = True)
    train_transform = count_category.fit_transform(train["category_name"])
    test_transform = count_category.transform(test['category_name'])
    return train_transform,test_transform

In [0]:
def brand_encoder(train,test):
    vect = LabelBinarizer(sparse_output=True)
    train_transform = vect.fit_transform(train["brand_name"])
    test_transform = vect.transform(test["brand_name"])
    return train_transform,test_transform

In [0]:
def ridge_model(X_train,y_train,params):
    model = Ridge(solver = "lsqr", fit_intercept=False,alpha = params)
    model.fit(X_train,y_train)
    return model

In [0]:
def build_mlp_model1(train_shape):
    model_in = Input(shape=(train_shape,), dtype='float32',sparse = True)
    out = Dense(256, activation='relu')(model_in)
    out = Dense(64, activation='relu')(out)
    out = Dense(64, activation='relu')(out)
    out = Dense(32, activation='relu')(out)
    model_out = Dense(1)(out)
    model = Model(model_in, model_out)
    return model

In [0]:
def build_mlp_model2(train_shape):
    model_in = Input(shape=(train_shape,), dtype='float32',sparse = True)
    out = Dense(1024, activation='relu')(model_in)
    out = Dense(512, activation='relu')(out)
    out = Dense(256, activation='relu')(out)
    out = Dense(128, activation='relu')(out)
    out = Dense(64, activation='relu')(out)
    out = Dense(32, activation='relu')(out)
    out = Dense(1)(out)
    model = Model(model_in, out)
    return model

In [0]:
def data_clean(df):
    df = df[(df.price >= 3) & (df.price <= 2000)]
    return df

In [0]:
def function1(X_input):
    
    # (1) Load Data Files from Local...
    df_train = pd.read_csv('train.tsv',sep = '\t')
    df_test = X_input
    print(df_train.shape,df_test.shape)
    print("(1) done")
    ##################################################################################
    
    # (2) Clean Data..
    df_train = data_clean(df_train)
    print("(2) done")
    ##################################################################################
    
    # (3) Encode Data..
    y_scaler = StandardScaler()
    y_train = y_scaler.fit_transform(np.log1p(df_train['price'].values.reshape(-1, 1)))
    df_train = data_preprocess(df_train)
    df_test = data_preprocess(df_test)
    train_name,test_name = text_encoder(df_train['name'],df_test['name'],'TFIDF',((1,1),1,1.0,100000))
    train_text,test_text = text_encoder(df_train['text'],df_test['text'],'TFIDF',((1,2),1,1.0,100000))
    train_dummies,test_dummies = dummy_encoder(df_train,df_test)
    X_train = scipy.sparse.hstack((train_name, train_text, train_dummies)).tocsr().astype('float32')
    X_test = scipy.sparse.hstack((test_name, test_text, test_dummies)).tocsr().astype('float32')
    ##################################################################################
    
    # (4) Train Models..
    ridge = ridge_model(X_train,y_train,10)
    preds = ridge.predict(X_test)[:, 0]
    preds_ridge = np.expm1(y_scaler.inverse_transform(preds.reshape(-1, 1))[:, 0])

    mlp1 = build_mlp_model1(X_train.shape[1])
    mlp1.compile(optimizer='adam', loss='mean_squared_error')
    for i in range(2):
        mlp1.fit(X_train,y_train,batch_size = 2**(8+i),epochs = 1,verbose = 1)
    preds = mlp1.predict(X_test,verbose = 1)[:, 0]
    preds_mlp1 = np.expm1(y_scaler.inverse_transform(preds.reshape(-1, 1))[:, 0])
    
    mlp2 = build_mlp_model2(X_train.shape[1])
    mlp2.compile(optimizer='adam', loss='mean_squared_error')
    for i in range(2):
        mlp2.fit(X_train,y_train,batch_size = 2**(8+i),epochs = 1,verbose = 1)
    preds = mlp2.predict(X_test,verbose = 1)[:, 0]
    preds_mlp2 = np.expm1(y_scaler.inverse_transform(preds.reshape(-1, 1))[:, 0])
    ###################################################################################################################
    
    # (4) Generate Best Ensemble
    preds_mlps = 0.55*preds_mlp1 + 0.45*preds_mlp2
    preds_f = 0.1*preds_ridge + 0.9*preds_mlps
    ###################################################################################################################
    
    # (6) Prediction
    df_test['prediction'] = preds_f
    return preds_f
    ###################################################################################################################

In [19]:
%%time
import warnings
warnings.filterwarnings("ignore")
import math
import time
import pickle
import pandas as pd
import numpy as np
import scipy
import scipy.sparse
from tqdm import tqdm,tqdm_notebook
from contextlib import contextmanager
import os
import re
import gc
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer,StandardScaler,OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import tensorflow.keras
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K

if __name__ == '__main__':
    
    # Reading Test File as Input
    gc.collect()
    df_test = pd.read_csv('test.tsv',sep = '\t')
    df_input = df_test[2:3]
    
    # Sending input to function1
    pred = function1(df_input)
    print('Predicted Price for ',df_input.values," is ",pred)


ERROR! Session/line number was not unique in database. History logging moved to new session 63
(1482535, 8) (1, 7)
(1) done
(2) done
1/1 [==============================] - 0s 2ms/step
Predicted Price for  [[2 'Coach bag Coach' 1 'Vintage & Collectibles/Bags and Purses/Handbag'
  'Coach' 1 'Brand new coach bag. Bought for [rm] at a Coach outlet.'
  'Brand new coach bag. Bought for [rm] at a Coach outlet. Coach bag Coach Vintage & Collectibles/Bags and Purses/Handbag']]  is  [49.917088]



#  <font color='red'>Function 2</font>

In [0]:
'''
This function takes 'df_test' and the 'actual price' as input and returns the RMSLE score

'df_test' is a data point takes the Inputs of 
["train_id" (index),
"name" (str),
"item_condition_id" (categorical),
"category_name" (str),
"brand_name" (str),
"shipping" (categorical),
"item_description" (str)]
'''

In [0]:
def rmsle_score(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [0]:
def function2(X,Y):
    score = rmsle_score(function1(X),Y)
    return score

In [22]:
%%time
import warnings
warnings.filterwarnings("ignore")
import math
import time
import pickle
import pandas as pd
import numpy as np
import scipy
import scipy.sparse
from tqdm import tqdm,tqdm_notebook
from contextlib import contextmanager
import os
import re
import gc
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer,StandardScaler,OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import tensorflow.keras
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K

if __name__ == '__main__':

      # Reading Test File as Input
      gc.collect()
      df_test = pd.read_csv('test.tsv',sep = '\t')
      df_input = df_test[2:3]
      actual_price = [30]
    
      score = function2(df_input,actual_price)
      print('RMSLE score is ',score)

(1482535, 8) (1, 7)
(1) done
(2) done
1/1 [==============================] - 0s 2ms/step
RMSLE score is  0.3231334323167663
CPU times: user 11min 13s, sys: 4min 22s, total: 15min 36s
Wall time: 15min 45s
